## Spatial Features Vs Urban Footprint Vs Population Size

Steps: 

    1) read zonal stats for spatial features (Independent vars) and GUF (Dependent var)
    2) merge the two files with id
    3) Locate the dependant and Independent variables (exclude columns like name)
    4) Standardize the dataset with μ=0 and σ=1  (using StandardScaler from sklearn)
    5) Run Pearson Correlation (Dependent vs Independent vars)
    6) Create a new dataset by selecting those independent variables with high statistical signficance 
    7) Split the new dataset into train(2/3) and test(1/3) sets
    7) Run Elastic Net with 10 fold cross validation
    
    

In [1]:
import sklearn
import pandas as pd
import numpy as np
import csv
import scipy.stats as stats
from statistics import pstdev
from statistics import mean
from sklearn import preprocessing
from sklearn.linear_model import ElasticNetCV
from sklearn.linear_model import ElasticNet, Lasso
from sklearn.datasets import make_regression
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from itertools import product
import copy
import geopandas as gpd
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae

In [2]:
np.random.seed(12345)

### Read the Features data

In [3]:
spfeas = pd.read_excel('sri_spfeas_v3.xlsx', sheet_name=0)

In [4]:
spfeas['gid'] = spfeas['gid'].astype(int)
spfeas = spfeas.set_index('gid')

In [5]:
spfeas.head()

,fid,gnd_n,gnd_c,fourier_sc31_mean_mean,fourier_sc31_mean_std,fourier_sc31_mean_sum,fourier_sc31_variance_mean,fourier_sc31_variance_std,fourier_sc31_variance_sum,fourier_sc51_mean_mean,...,sfs_sc71_min_ll_sum,sfs_sc71_roa_mean,sfs_sc71_roa_std,sfs_sc71_roa_sum,sfs_sc71_std_mean,sfs_sc71_std_std,sfs_sc71_std_sum,sfs_sc71_w_mean_mean,sfs_sc71_w_mean_std,sfs_sc71_w_mean_sum
gid,,,,,,,,,,,,,,,,,,,,,
43,0,Karagaskada South,2148065,69.453586,3.136141,2.376702e+05,18.629130,0.976240,6.374888e+04,78.493900,...,3933,136.394391,0.938602,466878,16.233955,3.648450,55568.828125,1.228705,0.707158,4205.858398
564,1,Panvila,2115070,59.535572,7.845283,1.400872e+06,17.086123,2.204723,4.020365e+05,69.651317,...,40076188,137.910516,2.093903,3245724,14.498129,5.118067,341213.468750,2.838843,3.471181,66812.164062
7917,2,Ganepalla,9227135,50.730112,5.682764,3.625884e+06,14.450501,2.109653,1.032835e+06,60.710447,...,7900571136,139.556597,2.014015,9974110,9.856942,7.202832,704475.625000,3.388618,5.864246,242184.515625
2263,3,Akkaraipattu 11,5236065,73.670361,1.821871,9.731855e+04,18.843218,1.231690,2.489189e+04,82.544710,...,1331,136.077853,0.267941,180031,10.558998,4.329045,13969.554688,0.514668,0.239074,680.906372
8,4,Delgasthenna,2109125,66.878083,2.710715,2.657735e+05,18.708531,1.015577,7.434770e+04,75.957167,...,4809,136.681028,1.377590,542487,15.218429,4.465160,60401.945312,1.180995,0.868924,4687.369141


### Load GUF

In [6]:
#guf = pd.read_csv('sri_guf.csv')
guf = gpd.read_file('../../../analysis/sri_lanka/shapefiles/sri_guf.shp')
guf['gid'] = guf['gid'].astype(int)
guf = guf.set_index('gid')
guf.head()

,gnd_n,gnd_c,prov_c,dis_c,dsd_c,guf_count,guf_sum,guf_mean,Shape_Area,geometry
gid,,,,,,,,,,
43,Karagaskada South,2148065,2,21,2148,2236.0,25755.0,11.518336,342335.19,"POLYGON ((80.70650072195758 7.176928435503455,..."
564,Panvila,2115070,2,21,2115,15373.0,101745.0,6.618422,2352362.54,"POLYGON ((80.73290623821237 7.36361987297315, ..."
7917,Ganepalla,9227135,9,92,9227,46726.0,0.0,0.000000,7147144.18,"POLYGON ((80.36778185838672 7.016079031081547,..."
2263,Akkaraipattu 11,5236065,5,52,5236,868.0,156060.0,179.792627,132541.12,"POLYGON ((81.86234055205938 7.22704322335137, ..."
8,Delgasthenna,2109125,2,21,2109,2599.0,113475.0,43.661023,397006.47,"POLYGON ((80.62702879794956 7.380156075748994,..."


### Merge population data with urbanfootprint and featuures

In [7]:
guf['PCNT_blt'] = ((guf['guf_sum']/255)*144)/(guf['Shape_Area'])*100


In [8]:
stats.describe(guf['PCNT_blt'])

DescribeResult(nobs=14021, minmax=(0.0, 95.02219828262784), mean=9.778191283584716, variance=416.63706895452043, skewness=2.5246161598633914, kurtosis=5.618559156477573)

In [9]:
colombo= guf[(guf['dis_c']==11) | (guf['dis_c']==12)| (guf['dis_c']==13)]


In [10]:
colombo.plot(column='PCNT_blt', cmap='OrRd',figsize=(10,8));


In [11]:
guf_cols =guf.drop(['geometry'],axis=1)
spfeas_guf = spfeas.merge(guf_cols, left_on='gid', right_on="gid", how='outer')

In [12]:
spfeas_guf = spfeas_guf.round(3)
spfeas_guf.head()

,fid,gnd_n_x,gnd_c_x,fourier_sc31_mean_mean,fourier_sc31_mean_std,fourier_sc31_mean_sum,fourier_sc31_variance_mean,fourier_sc31_variance_std,fourier_sc31_variance_sum,fourier_sc51_mean_mean,...,gnd_n_y,gnd_c_y,prov_c,dis_c,dsd_c,guf_count,guf_sum,guf_mean,Shape_Area,PCNT_blt
gid,,,,,,,,,,,,,,,,,,,,,
43,0,Karagaskada South,2148065,69.454,3.136,237670.172,18.629,0.976,63748.883,78.494,...,Karagaskada South,2148065,2,21,2148,2236.0,25755.0,11.518,342335.19,4.248
564,1,Panvila,2115070,59.536,7.845,1400872.000,17.086,2.205,402036.469,69.651,...,Panvila,2115070,2,21,2115,15373.0,101745.0,6.618,2352362.54,2.442
7917,2,Ganepalla,9227135,50.730,5.683,3625884.000,14.451,2.110,1032835.125,60.710,...,Ganepalla,9227135,9,92,9227,46726.0,0.0,0.000,7147144.18,0.000
2263,3,Akkaraipattu 11,5236065,73.670,1.822,97318.547,18.843,1.232,24891.891,82.545,...,Akkaraipattu 11,5236065,5,52,5236,868.0,156060.0,179.793,132541.12,66.491
8,4,Delgasthenna,2109125,66.878,2.711,265773.500,18.709,1.016,74347.703,75.957,...,Delgasthenna,2109125,2,21,2109,2599.0,113475.0,43.661,397006.47,16.141


### Filter Dataset by Builtup Surface. 

Select Rows where  builtup is greater than or equal to 10 percent

In [13]:
spfeas_guf.shape

(14021, 436)

### Analysis

Get the list of dependent variables from the DataFrame to store in list y_vars

In [14]:
#y_var = list(filter_by_builtup.axes[1])[440]

y_var = list(spfeas_guf.axes[1])[435]
y_var

'PCNT_blt'

Get a list of all independent variables from the DataFrame in list all_x

In [15]:
all_x = list(spfeas_guf.axes[1])[3:423]

#Check
#all_x

### Compute Coorelation of features with population density

Store all features with the least correlation (stat. significance (p < 0.05)) 

The Pearson correlation coefficient **measures the linear relationship
between two datasets.** Strictly speaking, Pearson's correlation requires
that each dataset be **normally distributed, and not necessarily zero-mean.**

Like other correlation coefficients, this one varies between -1 and +1
with 0 implying no correlation. Correlations of -1 or +1 imply an exact
linear relationship. Positive correlations imply that as x increases, so
does y. Negative correlations imply that as x increases, y decreases.

The p-value roughly indicates **the probability of an uncorrelated system**
producing datasets that have a Pearson correlation at least as extreme
as the one computed from these datasets. 

***The p-values are not entirely
reliable but are probably reasonable for datasets larger than 500 or so.***

In [16]:
spfeas_guf[y_var] = spfeas_guf[y_var].fillna(0)
spfeas_guf[y_var].isnull().values.any()

False

In [17]:
y_dict = {}
x = []

for x_var in all_x:
    
    #Calculate the Pearson statistics, 
    # returns the Pearson value and p value
    
    p = stats.pearsonr(spfeas_guf[x_var],spfeas_guf[y_var])
    
    
    #print (y_var + " , " + x_var + " , " +  str(p[0]) + " , " + str(p[1]))
    
    #If p < 0.05 append to list x
    if p[1] < 0.05:
        x.append([x_var,(p[0])])

#List x is made into a DataFrame 
# which is sorted by the absolute values of the Pearson values
x_df = pd.DataFrame(x,columns=["x_var","abs_r2"]).sort_values("abs_r2",ascending=False)


#The dependent variable dictionary is given an entry 
# where the key is the name of the dependent variable
# and the value is a list of top 200 most significant values

y_dict[y_var] = list(x_df["x_var"][0:200])
#y_dict[y_var]
#Print out each dependent variable and 
#the number of x values that remain to check completion


In [18]:
x_df.head(5)

,x_var,abs_r2
186,hog_sc7_kurtosis_mean,0.844397
174,hog_sc5_kurtosis_mean,0.809688
278,mean_sc3_variance_mean,0.795769
192,hog_sc7_skew_mean,0.785758
367,sfs_sc31_mean_mean,0.775555


In [19]:
#check 

for key in y_dict.keys():
    print(key,len(y_dict[key]))


PCNT_blt 200


### Correlation Significance

For each dependent variable y in the list of all dependent values, calibrate the model.
Add new key to the output dictionary where y is the dependent variable curently being processed and the values are empty for now

In [20]:
#Initialize the output dictionary, Y_D, 
# with each key being a dependent variable and the values being the results of the analyses

Y_D = {}

Y_D[y_var]={}

#Dictionary Models is used to store each result object for later use if needed

Models ={}

#Get independent variables from the variable dictionary and store in list x_vars
x_vars = y_dict[y_var]


vars_df = pd.DataFrame()

vars_df[y_var] = spfeas_guf[y_var]


for x in x_vars:
    vars_df[x] = spfeas_guf[x]

In [21]:
vars_df.head()

,PCNT_blt,hog_sc7_kurtosis_mean,hog_sc5_kurtosis_mean,mean_sc3_variance_mean,hog_sc7_skew_mean,sfs_sc31_mean_mean,hog_sc5_skew_mean,gabor_sc3_variance_mean,gabor_sc3_filter_2_mean,gabor_sc3_filter_14_mean,...,hog_sc5_mean_mean,ndvi_sc5_variance_std,sfs_sc51_max_ll_mean,orb_sc31_max_sum,lbpm_sc3_kurtosis_std,orb_sc71_mean_sum,orb_sc51_mean_sum,orb_sc31_mean_sum,orb_sc51_max_sum,sfs_sc31_mean_std
gid,,,,,,,,,,,,,,,,,,,,,
43,4.248,1232837.607,332095.664,3.930,5621.894,4.617,2818.422,1.111,1.094,1.095,...,0.111,0.009,40.416,1064,87.372,152.727,84.491,31.062,2305,2.648
564,2.442,582042.462,180522.707,2.222,3684.138,3.057,2076.286,0.754,0.746,0.737,...,0.109,0.009,34.022,2266,88.223,364.782,182.773,49.651,6806,2.920
7917,0.000,191063.090,89183.994,1.108,2482.967,1.646,1514.059,0.448,0.446,0.448,...,0.100,0.008,21.864,0,78.314,0.000,0.000,0.000,0,1.894
2263,66.491,1713686.816,424855.492,6.590,5412.451,6.650,3254.023,1.931,1.903,1.904,...,0.111,0.010,28.221,0,85.624,0.000,0.000,0.000,0,2.442
8,16.141,1341415.447,306488.279,3.319,4976.435,4.773,2506.209,1.041,1.036,1.048,...,0.110,0.012,37.865,1,101.747,50.518,7.391,0.023,271,2.928


### Scale/Normalize Data

In [22]:
#minmax_scaler = preprocessing.MinMaxScaler()
standard_scaler = preprocessing.StandardScaler()

names = vars_df.columns
scaled_df = standard_scaler.fit_transform(vars_df)
scaled_df = pd.DataFrame(scaled_df, columns=names)
scaled_df.head()

,PCNT_blt,hog_sc7_kurtosis_mean,hog_sc5_kurtosis_mean,mean_sc3_variance_mean,hog_sc7_skew_mean,sfs_sc31_mean_mean,hog_sc5_skew_mean,gabor_sc3_variance_mean,gabor_sc3_filter_2_mean,gabor_sc3_filter_14_mean,...,hog_sc5_mean_mean,ndvi_sc5_variance_std,sfs_sc51_max_ll_mean,orb_sc31_max_sum,lbpm_sc3_kurtosis_std,orb_sc71_mean_sum,orb_sc51_mean_sum,orb_sc31_mean_sum,orb_sc51_max_sum,sfs_sc31_mean_std
0,-0.270942,1.399884,1.444500,0.799661,1.531891,1.075406,1.509675,0.576564,0.557298,0.556021,...,0.391284,-0.155864,1.457226,-0.183476,0.599962,-0.188927,-0.170901,-0.130436,-0.231838,0.305300
1,-0.359424,-0.066859,-0.164669,-0.208227,-0.206404,-0.018677,-0.189176,-0.247507,-0.257276,-0.279333,...,-0.020762,-0.155864,0.291555,0.074860,0.764645,0.055598,0.042717,-0.002243,0.147865,0.906535
2,-0.479066,-0.948036,-1.134362,-0.865597,-1.283933,-1.008261,-1.476190,-0.953854,-0.959496,-0.953684,...,-1.874972,-0.501324,-1.924935,-0.412154,-1.152918,-0.365040,-0.354545,-0.344645,-0.426287,-1.361361
3,2.778546,2.483608,2.429280,2.369324,1.344007,2.501221,2.506825,2.469389,2.450949,2.443736,...,0.391284,0.189595,-0.766009,-0.412154,0.261694,-0.365040,-0.354545,-0.344645,-0.426287,-0.150048
4,0.311735,1.644594,1.172640,0.439111,0.952872,1.184814,0.794977,0.414981,0.421536,0.446352,...,0.185261,0.880513,0.992161,-0.411939,3.381773,-0.306787,-0.338480,-0.344486,-0.403425,0.924219


In [23]:
#X.head()

### Set Elastic net's parameters

In [24]:
enet_result = ElasticNetCV(max_iter=1e8,
                    alphas = [0.0005, 0.001, 0.01, 0.03, 0.05, 0.1],
                    l1_ratio =[.1, .5, .7, .9, .95, .99, 1],
                    verbose= False,
                    n_jobs = -1, 
                    cv=5, 
                    selection = 'random',
                    fit_intercept=False)


In [25]:
scaled_df.shape

(14021, 201)

In [26]:
# Fit the mode

In [27]:
#Fit the model with the scaled data
enet_result.fit(scaled_df[x_vars],scaled_df[y_var])
#Append the model to the Models dictionary
Models[y_var] = enet_result


In [28]:
enet_result

ElasticNetCV(alphas=[0.0005, 0.001, 0.01, 0.03, 0.05, 0.1], copy_X=True, cv=5,
       eps=0.001, fit_intercept=False,
       l1_ratio=[0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1], max_iter=100000000.0,
       n_alphas=100, n_jobs=-1, normalize=False, positive=False,
       precompute='auto', random_state=None, selection='random',
       tol=0.0001, verbose=False)

In [29]:
opt_alpha, opt_l1_ratio = enet_result.alpha_, enet_result.l1_ratio_

#Print update to ensure that the script is progressing properly
print("R2: {:.2f} Alpha: {} l1_ratio: {}"
      .format(enet_result.score(scaled_df[x_vars],scaled_df[y_var]),
              enet_result.alpha_, enet_result.l1_ratio_))

R2: 0.86 Alpha: 0.001 l1_ratio: 0.5


Record the overall R squared score and optimal alpha 
and l1 ratio values and store them in the output dictionary


In [30]:
Y_D[y_var]['Total R2'] = enet_result.score(scaled_df[x_vars],scaled_df[y_var])
Y_D[y_var]['Alpha'] = opt_alpha
Y_D[y_var]['l1_ratio'] = opt_l1_ratio

### Ten Fold Cross validated regression

In [31]:
#Create a list R2s to store out of sample R squared values

R2s = []

#Specify the number of trials to run

trials = 10

#Run the number of trials specified in trials, 
#for each trial 66% of the observations are randomly selected to train the model
#Testing is done on the remaining 33% of observations and the R squared values are recorded

for i in range(trials):    
    X_train, X_test, y_train, y_test = train_test_split(scaled_df[x_vars],scaled_df[y_var], test_size=0.34)
    enet_regr = ElasticNetCV(max_iter=1e8,
                    alphas = [opt_alpha],
                    l1_ratio =[opt_l1_ratio],
                    n_jobs = -1, 
                    cv=5, 
                    selection = 'random',
                    fit_intercept=False)
    y_pred = enet_regr.fit(X_train,y_train).predict(X_test)
    RMSE = np.sqrt(mse(y_test, y_pred))
    MAE = (mae(y_test, y_pred))
    r_squared = enet_regr.score(X_test,y_test)
    adjusted_r_squared = 1 - (1-r_squared)*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1)
    R2s.append(r_squared)

#print("Mean R2: {:.2f} StDev: {:.4f}".format(mean(R2s),pstdev(R2s)))
#Record the out of sample R squared values
Y_D[y_var]["Adj_R2"] = adjusted_r_squared
Y_D[y_var]["RMSE"] = RMSE
Y_D[y_var]["MAE"] = MAE
Y_D[y_var]['Sampling']={'trials':trials,'R2':mean(R2s),'StDev':pstdev(R2s),'R2s':R2s}
#coefs = [i for i in zip(list(scaled_df[x_vars].axes[1]),enet_result.coef_)]
#remaining = [i for i in coefs if abs(i[1])>0.0]
#Y_D[y_var]["Coefficients"]=remaining

In [32]:
Y_D[y_var]

{'Total R2': 0.8626770597619745,
 'Alpha': 0.001,
 'l1_ratio': 0.5,
 'Adj_R2': 0.8503920270502354,
 'RMSE': 0.38043204763141164,
 'MAE': 0.22685359833280613,
 'Sampling': {'trials': 10,
  'R2': 0.8583374088179804,
  'StDev': 0.004291823579051157,
  'R2s': [0.8602527544029451,
   0.8658310169347412,
   0.8567177581414632,
   0.8581513463344407,
   0.8602696390422984,
   0.8552876944057632,
   0.865263943493409,
   0.8545595296782367,
   0.8534143109714593,
   0.8536260947750466]}}

In [33]:
y_df = pd.DataFrame([i for i in zip(list(scaled_df[x_vars].axes[1]),enet_result.coef_)], 
                    columns=["features","Coeff"]).sort_values("Coeff", ascending=False)

y_df.head(25)

,features,Coeff
2,mean_sc3_variance_mean,0.473311
0,hog_sc7_kurtosis_mean,0.429980
4,sfs_sc31_mean_mean,0.390814
14,hog_sc3_kurtosis_mean,0.382340
26,lbpm_sc3_kurtosis_mean,0.251508
87,orb_sc71_max_mean,0.247130
21,ndvi_sc3_variance_mean,0.199201
190,hog_sc5_mean_mean,0.185399
89,orb_sc71_mean_mean,0.169621
23,lbpm_sc5_kurtosis_mean,0.160141


# Result

At national Scale, with GUF we can explain, 86% varriation of built up surface/human settlement



HOG, NDIV, SFS and MEAN are significant

## HOG 

In [34]:
filter_var = [col for col in scaled_df if col.startswith('hog')]
X_train, X_test, y_train, y_test = train_test_split(scaled_df[filter_var],scaled_df[y_var], test_size=0.34)
enet_result.fit(X_train,y_train)

Models[y_var] = enet_result

opt_alpha, opt_l1_ratio = enet_result.alpha_, enet_result.l1_ratio_


print("R2: {:.2f} Alpha: {} l1_ratio: {}"
      .format(enet_result.score(scaled_df[filter_var],scaled_df[y_var]),
              enet_result.alpha_, enet_result.l1_ratio_))


R2: 0.78 Alpha: 0.0005 l1_ratio: 0.1


## Gabor

In [35]:
filter_var = [col for col in scaled_df if col.startswith('gabor')]
X_train, X_test, y_train, y_test = train_test_split(scaled_df[filter_var],scaled_df[y_var], test_size=0.34)
enet_result.fit(X_train,y_train)

Models[y_var] = enet_result

opt_alpha, opt_l1_ratio = enet_result.alpha_, enet_result.l1_ratio_


print("R2: {:.2f} Alpha: {} l1_ratio: {}"
      .format(enet_result.score(scaled_df[filter_var],scaled_df[y_var]),
              enet_result.alpha_, enet_result.l1_ratio_))


R2: 0.71 Alpha: 0.0005 l1_ratio: 0.1


## Fourier

In [36]:
filter_var = [col for col in scaled_df if col.startswith('fourier')]
X_train, X_test, y_train, y_test = train_test_split(scaled_df[filter_var],scaled_df[y_var], test_size=0.34)
enet_result.fit(X_train,y_train)

Models[y_var] = enet_result

opt_alpha, opt_l1_ratio = enet_result.alpha_, enet_result.l1_ratio_


print("R2: {:.2f} Alpha: {} l1_ratio: {}"
      .format(enet_result.score(scaled_df[filter_var],scaled_df[y_var]),
              enet_result.alpha_, enet_result.l1_ratio_))


R2: 0.63 Alpha: 0.0005 l1_ratio: 1.0


## lbpm

In [37]:
filter_var = [col for col in scaled_df if col.startswith('lbpm')]
X_train, X_test, y_train, y_test = train_test_split(scaled_df[filter_var],scaled_df[y_var], test_size=0.34)
enet_result.fit(X_train,y_train)

Models[y_var] = enet_result

opt_alpha, opt_l1_ratio = enet_result.alpha_, enet_result.l1_ratio_


print("R2: {:.2f} Alpha: {} l1_ratio: {}"
      .format(enet_result.score(scaled_df[filter_var],scaled_df[y_var]),
              enet_result.alpha_, enet_result.l1_ratio_))


R2: 0.73 Alpha: 0.0005 l1_ratio: 1.0


In [38]:
filter_var = [col for col in scaled_df if col.startswith('mean')]
X_train, X_test, y_train, y_test = train_test_split(scaled_df[filter_var],scaled_df[y_var], test_size=0.34)
enet_result.fit(X_train,y_train)

Models[y_var] = enet_result

opt_alpha, opt_l1_ratio = enet_result.alpha_, enet_result.l1_ratio_


print("R2: {:.2f} Alpha: {} l1_ratio: {}"
      .format(enet_result.score(scaled_df[filter_var],scaled_df[y_var]),
              enet_result.alpha_, enet_result.l1_ratio_))


R2: 0.73 Alpha: 0.0005 l1_ratio: 0.1
